In [92]:
from BorderQuery import select_mungedata, select_features
from BorderModel import rolling_volume_aggregate, handle_categoricals, print_importances
import pandas as pd
import pandas
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import pdb

In [97]:
def run_rolling_mean_model(model, rv_days, munger_id, crossing_id, train_start, train_end, test_start, test_end):
    df_train = select_mungedata(munger_id, crossing_id, train_start, train_end)
    df_train = handle_categoricals(df_train, ['event'])
    df_test = select_mungedata(munger_id, crossing_id, test_start, test_end)
    df_test = handle_categoricals(df_test, ['event'], columns=df_train.columns.values)
#     pdb.set_trace()
    rv = rolling_volume_aggregate(rv_days)
    rv_train = rv[(rv.index >= train_start) & (rv.index < train_end)]
    rv_test = rv[(rv.index >= test_start) & (rv.index < test_end)]

    df_train = df_train.merge(rv_train, left_on=df_train.index.date, right_on=rv_train.index.date)
    df_test = df_test.merge(rv_test, left_on=df_test.index.date, right_on=rv_test.index.date)

    df_train = df_train.set_index('key_0')
    df_train.index.name = 'date'
    df_train.index = pandas.to_datetime(df_train.index)
    df_test = df_test.set_index('key_0')
    df_test.index.name = 'date'
    df_test.index = pandas.to_datetime(df_test.index)

    y_train = df_train.waittime
    X_train = df_train.drop('waittime', 1)
    y_test= df_test.waittime
    X_test = df_test.drop ('waittime', 1)

    model.fit(X_train, y_train)

#     print "OOB: ", model.oob_score_
    print "Test r2: ", model.score(X_test, y_test)
    print_importances(model, X_test.columns)
    
    return model.score(X_test, y_test)

In [65]:
import os
from ipyparallel import Client
rc = Client()
dview = rc[:]

# set proper working directory on all clients
cwd = os.getcwd()
dview.map(os.chdir, [cwd] * 32)
# print(dview.apply_sync(os.getcwd))

with dview.sync_imports():
    import datetime
    import pandas
    from BorderModel import print_importances, handle_categoricals, rolling_volume_aggregate
    from BorderQuery import select_features, select_mungedata
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.grid_search import GridSearchCV

importing datetime on engine(s)
importing pandas on engine(s)
importing print_importances,handle_categoricals,rolling_volume_aggregate from BorderModel on engine(s)
importing select_features,select_mungedata from BorderQuery on engine(s)
importing ExtraTreesRegressor from sklearn.ensemble on engine(s)
importing GridSearchCV from sklearn.grid_search on engine(s)


In [4]:
import random

def create_train_test(year, train_length=2):
    '''
    IN 
        years: list of years to predict
        train_length: number of years to train
    '''
    train_start = datetime.date(year - train_length, 1, 1).strftime('%Y-%m-%d')
    train_end = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    test_start = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    test_end = datetime.date(year + 1, 1, 1).strftime('%Y-%m-%d')
    return train_start, train_end, test_start, test_end
    
def compare_years_parallel(model, rv_days, xing, munger_id, years):
    prlist = {}
    for year in years:
        cpu = random.randint(0, 31)
        train_start, train_end, test_start, test_end = create_train_test(year, 2)

        prlist[year] = rc[cpu].apply_async(run_rolling_mean_model, model, rv_days, munger_id, xing,  
                                           train_start, train_end, 
                                           test_start, test_end)
        
    return prlist

In [60]:
train_start = '2013-1-1'
train_end = '2015-1-1'
test_start = train_end
test_end = '2016-1-1'
df_train = select_mungedata(3, 1, train_start, train_end)
df_train = handle_categoricals(df_train, ['event'])
df_test = select_mungedata(3, 1, test_start, test_end)
df_test = handle_categoricals(df_test, ['event'])

In [62]:
rv = rolling_volume_aggregate([1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35])
rv_train = rv[(rv.index >= train_start) & (rv.index < train_end)]
rv_test = rv[(rv.index >= test_start) & (rv.index < test_end)]

In [64]:
df_train = df_train.merge(rv_train, left_on=df_train.index.date, right_on=rv_train.index.date)
df_test = df_test.merge(rv_test, left_on=df_test.index.date, right_on=rv_test.index.date)

In [66]:
df_train = df_train.set_index('key_0')
df_train.index.name = 'date'
df_train.index = pd.to_datetime(df_train.index)

,waittime,year,month,week,dayofweek,minofday,temp_max,temp_mean,temp_min,viz_max,...,vol_mean_2,vol_mean_3,vol_mean_4,vol_mean_5,vol_mean_6,vol_mean_7,vol_mean_14,vol_mean_21,vol_mean_28,vol_mean_35
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.000000,2013,1,1,1,0,39,32,26,10,...,-2.370207,-1.677587,-1.108272,-0.162907,0.174595,0.771929,0.108722,-0.471445,-0.615379,-0.403825
2013-01-01,0.000000,2013,1,1,1,30,39,32,26,10,...,-2.370207,-1.677587,-1.108272,-0.162907,0.174595,0.771929,0.108722,-0.471445,-0.615379,-0.403825
2013-01-01,0.133515,2013,1,1,1,60,39,32,26,10,...,-2.370207,-1.677587,-1.108272,-0.162907,0.174595,0.771929,0.108722,-0.471445,-0.615379,-0.403825
2013-01-01,0.120116,2013,1,1,1,90,39,32,26,10,...,-2.370207,-1.677587,-1.108272,-0.162907,0.174595,0.771929,0.108722,-0.471445,-0.615379,-0.403825
2013-01-01,0.137763,2013,1,1,1,120,39,32,26,10,...,-2.370207,-1.677587,-1.108272,-0.162907,0.174595,0.771929,0.108722,-0.471445,-0.615379,-0.403825


In [67]:
df_test = df_test.set_index('key_0')
df_test.index.name = 'date'
df_test.index = pd.to_datetime(df_test.index)

In [68]:
y_train = df_train.waittime
X_train = df_train.drop('waittime', 1)
y_test= df_test.waittime
X_test = df_test.drop ('waittime', 1)

In [69]:
model = RandomForestRegressor(n_jobs=-1, n_estimators=16, oob_score=True)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=16, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [70]:
model.oob_score_

0.88705005946983073

In [71]:
model.score(X_test, y_test)

0.070903088573312054

In [73]:
print_importances(model, X_train.columns)

[   ('minofday', 0.54686486797478207),
    ('dayofweek', 0.078016583424563662),
    ('vol_mean_1', 0.036317663508840144),
    ('week', 0.030749761949966918),
    ('vol_mean_28', 0.011828172483873842),
    ('vol_mean_35', 0.01180972953433318),
    ('temp_mean_p3', 0.011273246933643448),
    ('temp_mean_p2', 0.010542637411584874),
    ('vol_mean_2', 0.010442175831818053),
    ('vol_mean_3', 0.0097942332838892917),
    ('vol_mean_4', 0.0091680387152375947),
    ('vol_mean_14', 0.0091195386665086815),
    ('temp_min_m2', 0.0089827649638900935),
    ('vol_mean_21', 0.0086664365077472882),
    ('temp_min_m1', 0.0085442670588822683),
    ('vol_mean_7', 0.0085418856630021679),
    ('vol_mean_5', 0.0085002324526534858),
    ('wind_max', 0.008467984791776054),
    ('month', 0.0080670567630121062),
    ('temp_max_m2', 0.007707233100664285),
    ('temp_min', 0.0064756230351935033),
    ('vol_mean_6', 0.0064134256413055325),
    ('temp_mean_m2', 0.0063473824099934545),
    ('temp_max_m1', 0.0063130

In [75]:
model2 = ExtraTreesRegressor(n_jobs=-1, n_estimators=96, bootstrap=True, oob_score=True)
model2.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [76]:
model2.oob_score_

0.9190880151251507

In [77]:
model2.score(X_test, y_test)

0.19262650175538387

In [78]:
print_importances(model, X_train.columns)

[   ('minofday', 0.54686486797478207),
    ('dayofweek', 0.078016583424563662),
    ('vol_mean_1', 0.036317663508840144),
    ('week', 0.030749761949966918),
    ('vol_mean_28', 0.011828172483873842),
    ('vol_mean_35', 0.01180972953433318),
    ('temp_mean_p3', 0.011273246933643448),
    ('temp_mean_p2', 0.010542637411584874),
    ('vol_mean_2', 0.010442175831818053),
    ('vol_mean_3', 0.0097942332838892917),
    ('vol_mean_4', 0.0091680387152375947),
    ('vol_mean_14', 0.0091195386665086815),
    ('temp_min_m2', 0.0089827649638900935),
    ('vol_mean_21', 0.0086664365077472882),
    ('temp_min_m1', 0.0085442670588822683),
    ('vol_mean_7', 0.0085418856630021679),
    ('vol_mean_5', 0.0085002324526534858),
    ('wind_max', 0.008467984791776054),
    ('month', 0.0080670567630121062),
    ('temp_max_m2', 0.007707233100664285),
    ('temp_min', 0.0064756230351935033),
    ('vol_mean_6', 0.0064134256413055325),
    ('temp_mean_m2', 0.0063473824099934545),
    ('temp_max_m1', 0.0063130

## Wow, best model yet, but only useful for short term estimates since unable to predict volume

In [87]:
train_start = '2013-1-1'
train_end = '2015-1-1'
test_start = train_end
test_end = '2016-1-1'
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.919740160075
Test r2:  0.19776394358
[   ('minofday', 0.55531089458098282),
    ('dayofweek', 0.076425696394780016),
    ('month', 0.012492590615103209),
    ('week', 0.012227323981350935),
    ('vol_mean_1', 0.012081821125198787),
    ('year', 0.0097656272445090283),
    ('vol_mean_2', 0.0083851824634064657),
    ('vol_mean_49', 0.0081786242053452606),
    ('event_goodfriday', 0.0076872461844161204),
    ('vol_mean_3', 0.0076743682299256289),
    ('temp_min_m2', 0.0074395885940774778),
    ('temp_min_m1', 0.0067429795437263006),
    ('temp_mean_m2', 0.0067423350281688582),
    ('temp_mean_p3', 0.006583696476610639),
    ('temp_mean_p2', 0.0063092498509102314),
    ('temp_min', 0.006221912193521772),
    ('vol_mean_42', 0.0060078560155279849),
    ('temp_min_p3', 0.0058783187281200083),
    ('temp_mean', 0.0058189329642338748),
    ('vol_mean_28', 0.0057095305081166945),
    ('temp_mean_m1', 0.0056454939988019029),
    ('vol_mean_35', 0.005597212539134898),
    ('vol_mean_21', 

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [90]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.917915445067
Test r2:  0.183807058206
[   ('minofday', 0.55988777189187411),
    ('dayofweek', 0.074783368690708063),
    ('week', 0.013025344142966947),
    ('vol_mean_1', 0.012943549700483078),
    ('month', 0.012850118816383668),
    ('year', 0.010969854717235786),
    ('vol_mean_2', 0.0086262459433350605),
    ('vol_mean_3', 0.0081888159043456647),
    ('temp_min_m2', 0.0080107927402941325),
    ('event_goodfriday', 0.0075469973493971394),
    ('temp_mean_m2', 0.0074866840427289119),
    ('vol_mean_28', 0.0073505278074067759),
    ('temp_min_m1', 0.0071664730533175297),
    ('temp_min', 0.0069333919757203219),
    ('vol_mean_21', 0.0065910138570475889),
    ('temp_mean_p3', 0.0065439849109754054),
    ('temp_mean_p2', 0.0064176482139738237),
    ('temp_min_p3', 0.0064078643190725081),
    ('temp_max_p1', 0.0061355353296366549),
    ('temp_mean', 0.0059242507073277058),
    ('temp_mean_p1', 0.0059223012680393776),
    ('temp_max_m2', 0.0058910579662559579),
    ('vol_mean_4'

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [89]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.920829597963
Test r2:  0.191351911854
[   ('minofday', 0.56030957382248558),
    ('dayofweek', 0.073770301449758463),
    ('week', 0.012493164073165806),
    ('vol_mean_1', 0.012416318964404002),
    ('month', 0.0122553680010018),
    ('year', 0.010268493561817128),
    ('vol_mean_2', 0.0092419228908607407),
    ('vol_mean_3', 0.0079627879373889172),
    ('temp_min_m2', 0.007659020796471982),
    ('temp_min_m1', 0.0076534891876159162),
    ('event_goodfriday', 0.0074627816240605965),
    ('temp_mean_p3', 0.0069477299610980143),
    ('vol_mean_42', 0.0069315023362919066),
    ('temp_mean_p2', 0.006317841818660009),
    ('temp_mean_m2', 0.0062051867952315305),
    ('temp_min', 0.0061824968067392691),
    ('temp_min_p3', 0.0061345782179776402),
    ('vol_mean_28', 0.0060365146566882842),
    ('temp_mean', 0.0058450490967801028),
    ('vol_mean_35', 0.0058084076394560704),
    ('temp_max_m2', 0.0057918584568337553),
    ('vol_mean_4', 0.0057478580996780729),
    ('temp_min_p2', 0.0

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [91]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.919219985784
Test r2:  0.21438265827
[   ('minofday', 0.55407279038961166),
    ('dayofweek', 0.077490242567601578),
    ('month', 0.011566162794839562),
    ('vol_mean_1', 0.010892401787429409),
    ('week', 0.010757093962071215),
    ('year', 0.0089346752480780108),
    ('vol_mean_2', 0.0088173447069015125),
    ('event_goodfriday', 0.0081125188575525854),
    ('vol_mean_3', 0.00795437990984783),
    ('vol_mean_63', 0.007809248954510341),
    ('temp_min_m2', 0.0073313738479717123),
    ('temp_min_m1', 0.0071509313505145828),
    ('temp_mean_m2', 0.0066584276689777983),
    ('vol_mean_56', 0.0064869443816923389),
    ('vol_mean_49', 0.0062892549959889113),
    ('temp_mean_p3', 0.0061843092466523511),
    ('temp_min_p3', 0.0058706749612073275),
    ('vol_mean_4', 0.0058007147913925576),
    ('temp_max_m2', 0.005763066231332094),
    ('temp_min', 0.0056621729065943717),
    ('temp_mean_p2', 0.0055711664756782032),
    ('vol_mean_42', 0.0053952268045137737),
    ('temp_mean_p1', 

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [92]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.922735198713
Test r2:  0.250495329346
[   ('minofday', 0.5597642934404673),
    ('dayofweek', 0.072705133832095994),
    ('vol_mean_1', 0.01328869112043478),
    ('vol_mean_77', 0.011957703063647242),
    ('month', 0.01028256855412338),
    ('week', 0.0095506811629874622),
    ('vol_mean_70', 0.0093060309991690408),
    ('year', 0.0083188722112617925),
    ('vol_mean_3', 0.0080721683511107388),
    ('event_goodfriday', 0.0076411362421150951),
    ('vol_mean_2', 0.007556963575990513),
    ('temp_min_m2', 0.0064804614234486987),
    ('temp_min_m1', 0.0064544778495270799),
    ('vol_mean_63', 0.0064156983245233267),
    ('temp_mean_m2', 0.0060706083728475184),
    ('temp_mean_p2', 0.0055589786535585909),
    ('temp_mean_p3', 0.0055275923145675669),
    ('vol_mean_4', 0.005308214661224094),
    ('temp_min', 0.0052233534861364428),
    ('vol_mean_49', 0.0051569796380661146),
    ('temp_mean_p1', 0.0050547104256453337),
    ('temp_min_p3', 0.0050517547662784767),
    ('wind_max', 0.0

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [93]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.921422143035
Test r2:  0.25543910489
[   ('minofday', 0.55899368880862932),
    ('dayofweek', 0.075452770347377018),
    ('vol_mean_1', 0.011301113906163623),
    ('vol_mean_77', 0.010898376498344145),
    ('month', 0.0091869609295038308),
    ('vol_mean_2', 0.0087104835985312625),
    ('vol_mean_84', 0.0086387937268823083),
    ('week', 0.0080192316433177809),
    ('vol_mean_3', 0.0079077460231188647),
    ('vol_mean_70', 0.0078910237945334569),
    ('event_goodfriday', 0.007484672342932857),
    ('year', 0.0073921738899002326),
    ('vol_mean_91', 0.0070279256123067938),
    ('temp_min_m2', 0.0062735087197758705),
    ('temp_min_m1', 0.0062416529611513437),
    ('temp_mean_m2', 0.0056630822029240168),
    ('vol_mean_63', 0.0054115146064083429),
    ('vol_mean_4', 0.0053197336274247485),
    ('temp_mean_p3', 0.0052641952429588051),
    ('temp_max_m2', 0.0050518713225032397),
    ('vol_mean_49', 0.0049984546176935025),
    ('vol_mean_21', 0.0049713375546767032),
    ('temp_min_

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [94]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.921764928622
Test r2:  0.266502887528
[   ('minofday', 0.55966773900639155),
    ('dayofweek', 0.07480542393632135),
    ('vol_mean_1', 0.011599766595977707),
    ('vol_mean_77', 0.0093957168620445626),
    ('vol_mean_2', 0.0089073273823491606),
    ('vol_mean_70', 0.0086104953151035814),
    ('event_goodfriday', 0.0079742971738145617),
    ('month', 0.0075746696039166997),
    ('week', 0.0074422322666579431),
    ('vol_mean_105', 0.0073596932107971502),
    ('vol_mean_84', 0.0073345713700223164),
    ('vol_mean_3', 0.0070871193241840063),
    ('year', 0.0069141663240720125),
    ('vol_mean_91', 0.0060961621486534103),
    ('temp_min_m2', 0.0060664367776290321),
    ('temp_min_m1', 0.0055875455164353903),
    ('vol_mean_98', 0.0055268602113903277),
    ('vol_mean_63', 0.0054930431671400834),
    ('temp_min', 0.0050520591757950714),
    ('vol_mean_49', 0.0049940147975314321),
    ('vol_mean_4', 0.0049314177639518285),
    ('vol_mean_21', 0.0048185714676383046),
    ('temp_mean_m

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [95]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.922036881045
Test r2:  0.269315198016
[   ('minofday', 0.56055647009847032),
    ('dayofweek', 0.074600217602254507),
    ('vol_mean_1', 0.01112352804741963),
    ('vol_mean_77', 0.0091059580056262589),
    ('vol_mean_3', 0.0082903441323230551),
    ('event_goodfriday', 0.0077799434418444439),
    ('vol_mean_2', 0.0076542527980540676),
    ('month', 0.0076376147931033382),
    ('week', 0.0075468113091089646),
    ('vol_mean_70', 0.0073940647654360189),
    ('vol_mean_84', 0.0065139021832206654),
    ('year', 0.0062999997835104004),
    ('temp_min_m1', 0.0057858782548518084),
    ('temp_min_m2', 0.0057612841083084015),
    ('vol_mean_105', 0.0057388473703305354),
    ('vol_mean_91', 0.0053824769182804237),
    ('vol_mean_98', 0.0053397571395495695),
    ('vol_mean_112', 0.0053136869557152376),
    ('vol_mean_119', 0.0052649215444778123),
    ('vol_mean_4', 0.0051454619073408141),
    ('vol_mean_63', 0.0051034690876206089),
    ('vol_mean_21', 0.0047359669575450011),
    ('wind_m

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [96]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119,
                               126, 133], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.923296402644
Test r2:  0.262708360474
[   ('minofday', 0.55585995164642032),
    ('dayofweek', 0.074070640272401983),
    ('vol_mean_1', 0.011742533518707505),
    ('vol_mean_77', 0.0094074100628057152),
    ('vol_mean_2', 0.0086242126434249271),
    ('vol_mean_3', 0.0085307830765262316),
    ('week', 0.007672981431958009),
    ('event_goodfriday', 0.007513788605796135),
    ('month', 0.0074762153364168063),
    ('vol_mean_70', 0.0071686826268269237),
    ('vol_mean_84', 0.0070415035283904895),
    ('year', 0.0057450008013011956),
    ('temp_min_m1', 0.0056692055090923317),
    ('temp_min_m2', 0.0054876271237208479),
    ('vol_mean_105', 0.0054221870133018485),
    ('vol_mean_4', 0.0052362930674403054),
    ('vol_mean_91', 0.0052238773077206155),
    ('vol_mean_112', 0.0051668941835574479),
    ('vol_mean_63', 0.0051019672942566015),
    ('vol_mean_49', 0.0048664531471685766),
    ('temp_max_p1', 0.0046948813884474614),
    ('temp_mean_m2', 0.0046797160832757513),
    ('vol_mea

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [97]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119,
                               126, 133, 140, 147], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.922411809042
Test r2:  0.234647180997
[   ('minofday', 0.55843210637799967),
    ('dayofweek', 0.073242618028852832),
    ('vol_mean_1', 0.011484235461128834),
    ('vol_mean_77', 0.0090369435374623181),
    ('vol_mean_2', 0.0088806840956022187),
    ('event_goodfriday', 0.0075198316082802745),
    ('month', 0.0075100973784343042),
    ('vol_mean_3', 0.0069614171782258682),
    ('week', 0.0067711952041667814),
    ('vol_mean_70', 0.0066583733115795065),
    ('vol_mean_84', 0.0064438656875207799),
    ('temp_min_m2', 0.0056141473498609422),
    ('vol_mean_4', 0.0053357303531543646),
    ('temp_mean_p3', 0.0053256422239494087),
    ('year', 0.005177886540004153),
    ('vol_mean_63', 0.0050664136421511826),
    ('vol_mean_98', 0.004933715579105929),
    ('temp_min_m1', 0.0049256816981973102),
    ('temp_mean_p2', 0.0049024342943569752),
    ('vol_mean_91', 0.0049007706339961495),
    ('vol_mean_105', 0.0048134358807837363),
    ('temp_mean_m2', 0.004765722433354937),
    ('vol_mea

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

In [98]:
run_rolling_mean_model(model2, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119,
                               126, 133, 140, 147, 154, 161], 
                       3, 1, train_start, train_end, test_start, test_end)

OOB:  0.922437253098
Test r2:  0.229842736433
[   ('minofday', 0.55791952132369715),
    ('dayofweek', 0.075066693658605035),
    ('vol_mean_1', 0.010772947982860916),
    ('vol_mean_77', 0.0084900482001964393),
    ('event_goodfriday', 0.0077062470338671099),
    ('vol_mean_2', 0.0075894653763237422),
    ('vol_mean_70', 0.0074024538646036524),
    ('vol_mean_3', 0.0070841057335740776),
    ('week', 0.0069853363082712852),
    ('month', 0.0068059616105664794),
    ('vol_mean_84', 0.0065387449815583086),
    ('temp_min_m2', 0.0057906812530802007),
    ('temp_min_m1', 0.0055347478251110867),
    ('vol_mean_4', 0.0051279017318941978),
    ('year', 0.005038972222860108),
    ('temp_mean_p2', 0.0048765473979547084),
    ('vol_mean_63', 0.0047620676939148466),
    ('temp_mean_m2', 0.0047206923945769261),
    ('vol_mean_49', 0.0046803007532300973),
    ('vol_mean_105', 0.0046599462226223414),
    ('temp_mean_p3', 0.0045853047350150758),
    ('vol_mean_91', 0.0044582939385610669),
    ('vol_m

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=96, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False)

## Volume rolling means out to 119 days yields r2 for 2015 of .269

# Compare model across years

In [98]:
# train_start, train_end, test_start, test_end = create_train_test(2014, 2)
# run_rolling_mean_model(model, [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119,
#                                126, 133], 
#                        3, 1, train_start, train_end, test_start, test_end)

Test r2:  0.578059971555
[   ('minofday', 0.59511579873119558),
    ('dayofweek', 0.071388243661709691),
    ('week', 0.01260759269283545),
    ('month', 0.010843860221282718),
    ('vol_mean_1', 0.0090394014701417318),
    ('temp_mean_m2', 0.006849680891436314),
    ('vol_mean_2', 0.0066196994592622148),
    ('vol_mean_3', 0.0060217384073494732),
    ('vol_mean_77', 0.0058315072102847129),
    ('vol_mean_84', 0.0057473607589563014),
    ('temp_min_m2', 0.0057464002401541184),
    ('temp_min', 0.0055914430951547476),
    ('temp_mean_m1', 0.0050915578140986141),
    ('vol_mean_4', 0.0048835250884830018),
    ('vol_mean_91', 0.0048592244374011063),
    ('temp_min_p1', 0.004818041206194835),
    ('temp_min_m1', 0.0048027377803349794),
    ('wind_max', 0.0046782704232081734),
    ('vol_mean_5', 0.004562804467462332),
    ('vol_mean_70', 0.0044937994657845556),
    ('temp_mean_p3', 0.0043816662979483528),
    ('event_goodfriday', 0.0042837471092742304),
    ('temp_min_p3', 0.004276914689275

0.5780599715548258

### Peace Arch

In [99]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
rv_days = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119]
pr1 = compare_years_parallel(model, rv_days, 1, 3, range(2011, 2016))

In [112]:
rm1 = {}
for year in range(2011, 2016):
#     print year, pr1[year].ready()
    if pr1[year].ready():
        rm1[year] = pr1[year].get(1)
        print year, rm1[year]

2011 0.0642442931581
2012 -1.63006223943
2013 0.614762001191
2014 0.58150484148
2015 0.264083570027


### Pacific Highway

In [107]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=96)
rv_days = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119]
pr5 = compare_years_parallel(model, rv_days, 5, 3, range(2011, 2016))

In [113]:
rm5 = {}
for year in range(2011, 2016):
#     print year, pr5[year].ready()
    if pr5[year].ready():
        rm5[year] = pr5[year].get(1)
        print year, rm5[year]

2011 0.57706807755
2012 0.355411279984
2013 0.613825545621
2014 0.5610390111
2015 0.396145081352
